#3. IMPORTING AND TIDYING MOVIES DATA

##3.01 Getting started.

We will use ujson library to import all json files with information of movies, which seems to be the most efficient and quickest way to import a big number of json files. Reference: [ujson](https://pypi.org/project/ujson/).

In [0]:
#Importing basic libraries:
import pandas as pd
import numpy as np
import timeit
import ujson
import os

Getting access to my Google Drive where jsons files have been compressed into a zip folder:

More information about it in https://colab.research.google.com/notebooks/io.ipynb and https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html.

This code is specific for being executed in Google Colab. A link will be showed; you will have to click on it, copy the authorization code and paste it on the box.

In [0]:
# Install the PyDrive wrapper & import libraries. This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client. This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#Getting movies.zip:
file_id = '1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("movies.zip")

Now movies.zip is availble in Gogle Colab virtual machine:

In [0]:
!ls -lh

total 1.5G
-rw-r--r-- 1 root root 2.5K Nov 17 17:55 adc.json
drwx------ 3 root root 4.0K Nov 17 17:58 drive
-rw-r--r-- 1 root root 1.5G Nov 17 21:15 movies.zip
drwxr-xr-x 2 root root 4.0K Nov 15 19:33 sample_data
drwxr-xr-x 3 root root 4.0K Nov 17 21:15 themoviedb_data
-rw-r--r-- 1 root root 6.6K Nov 17 21:51 typesCheck.xlsx


Let's unzip it to get acces to json files:

In [0]:
#We will get an error message. No worries about it!
!mkdir themoviedb_data
!cd themoviedb_data && unzip -q ../movies.zip
!ls -hl themoviedb_data

warning [../movies.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [../movies.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #409792).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)
total 12M
drwxr-xr-x 2 root root 12M Nov 17 21:17 movies


Now we have available our 409,791 files (one per movie):

In [0]:
!ls -1 themoviedb_data/movies | wc -l

409791


On the other hand, in some situations it will be useful to have access to our own Google Drive account to transfer files. We can get this by executing the following code:

In [0]:
#Again: click on the link, copy the authorization code and paste it on the box.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Now we have access to our own Google Drive account:
!ls -lh "/content/drive/My Drive/"

total 3.2G
drwx------ 2 root root 4.0K Jul 14 07:08  20180714_Deep_Learning
-rw------- 1 root root  76K Oct 22 18:13  6885.json
drwx------ 2 root root 4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
-rw------- 1 root root 2.7M Oct 28 21:03  Cast_Crew_ids.csv
drwx------ 2 root root 4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root  44K Feb  4  2018  contacts.vcf
-rw------- 1 root root    1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root  50K Nov  9 13:01  genres.xlsx
-rw------- 1 root root 1.5M Oct 24 17:34  IMDB_5000_movie.csv
-rw------- 1 root root  12K Nov  3 10:33  InflationConverter.xlsx
-rw------- 1 root root    1 Nov 10 10:21 'Mis cuentas.gsheet'
-rw------- 1 root root 117K Oct 28 21:04  Movies_ids.csv
-rw------- 1 root root 2.2M Nov 16 17:17  MoviesUS.csv
-rw------- 1 root root 1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root    1 Nov 17 16:27 'Nuestras Cuentas.gsheet'
-rw------- 1 root root 1.8G Oct 27 13:06  people.zip
-rw------- 1 root root 6.4

## 3.02 Checking json files structure.

Lets import one of them to check how it looks:

In [0]:
jsonTrial = ujson.load(open(r"./themoviedb_data/movies/2.json"))

In [0]:
#We can see what the keys are and whan kind of objects the values are:
pd.DataFrame([(x, type(jsonTrial[x])) for x in jsonTrial.keys()])

,0,1
0,adult,<class 'bool'>
1,backdrop_path,<class 'str'>
2,belongs_to_collection,<class 'NoneType'>
3,budget,<class 'int'>
4,genres,<class 'list'>
5,homepage,<class 'NoneType'>
6,id,<class 'int'>
7,imdb_id,<class 'str'>
8,original_language,<class 'str'>
9,original_title,<class 'str'>


Some of the values are dictionaries again. It would be interesting to know their keys. We can do it with this script, which builts **`dfTypeCheck`** dataframe with a summary of our json file structure:

In [0]:
#'key1' shows the first level of keys. And when key1[value] is a dictionary, key2 shows their keys.
#'aux' will be useful to count later on.
dfTypeCheck = pd.DataFrame([(x, 'NotDict', str(type(jsonTrial[x])), 'aux') for x in jsonTrial.keys() if not isinstance(jsonTrial[x], dict)], columns = ['key1', 'key2', 'type', 'aux'])
dfTypeCheck = dfTypeCheck.append(pd.DataFrame([(x, y, str(type(jsonTrial[x][y])), 'aux') for x in jsonTrial.keys() if isinstance(jsonTrial[x], dict) for y in jsonTrial[x].keys()], 
                                      columns = ['key1', 'key2', 'type', 'aux']))
dfTypeCheck

,key1,key2,type,aux
0,adult,NotDict,<class 'bool'>,aux
1,backdrop_path,NotDict,<class 'str'>,aux
2,belongs_to_collection,NotDict,<class 'NoneType'>,aux
3,budget,NotDict,<class 'int'>,aux
4,genres,NotDict,<class 'list'>,aux
5,homepage,NotDict,<class 'NoneType'>,aux
6,id,NotDict,<class 'int'>,aux
7,imdb_id,NotDict,<class 'str'>,aux
8,original_language,NotDict,<class 'str'>,aux
9,original_title,NotDict,<class 'str'>,aux


Now let's create a function to do the same with all our jsons, counting how many we have in every type:

In [0]:
def formatCheck(path):
    dfTypeCheck = pd.DataFrame()
    dfAux = pd.DataFrame()
    intI = 0

    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:               
                    data = ujson.load(o)
                    dfAux = dfAux.append(pd.DataFrame([(x, 'NotDict', str(type(data[x])), 'aux') for x in data.keys() if not isinstance(data[x], dict)], 
                                                              columns = ['key1', 'key2', 'type', 'aux']))
                    dfAux = dfAux.append(pd.DataFrame([(x, y, str(type(data[x][y])), 'aux') for x in data.keys() if isinstance(data[x], dict) 
                                                   for y in data[x].keys()], columns = ['key1', 'key2', 'type', 'aux']))
                    #Every 1000 files, we group and count (this is more efficient than doing it with every row!)
                    #And print the number of files (just to control how it goes). 
                    if intI % 1000 == 0:
                        dfTypeCheck = dfTypeCheck.append(dfAux.groupby(['key1','key2','type']).count())
                        dfAux = pd.DataFrame()
                        print(intI)
            
    #Last grouping:
    if intI % 1000 != 0:
        dfTypeCheck = dfTypeCheck.append(dfAux.groupby(['key1','key2','type']).count())
        dfAux = pd.DataFrame()

    #And now we group and sum all our counts:
    dfTypeCheck = dfTypeCheck.groupby(['key1','key2','type']).sum()
    
    return dfTypeCheck

In [0]:
#Let's launch the function to get dfTypeCheck with the structure and counts of all our json files.
#Just to see how it goes, we included in formatCheck function a counter every 1000 files.
tic=timeit.default_timer()
path = "./themoviedb_data/movies"
dfTypeCheck = pd.DataFrame()
dfTypeCheck = formatCheck(path)
toc=timeit.default_timer()
print((toc - tic)/60, ' min') #elapsed time in minutes

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

2074.924305291999

And so we get **`dfTypeCheck`** which shows a summary of the structure of our jsons files. This will be useful to define the information we will import from them.

In [0]:
dfTypeCheck

aux
key1                  key2          type                      
adult                 NotDict       <class 'bool'>      409791
alternative_titles    titles        <class 'list'>      409791
backdrop_path         NotDict       <class 'NoneType'>  270769
                                    <class 'str'>       139022
belongs_to_collection NotDict       <class 'NoneType'>  398774
                      backdrop_path <class 'NoneType'>    4364
                                    <class 'str'>         6653
                      id            <class 'int'>        11017
                      name          <class 'str'>        11017
                      poster_path   <class 'NoneType'>    2647
                                    <class 'str'>         8370
budget                NotDict       <class 'int'>       409791
credits               cast          <class 'list'>      409791
                      crew          <class 'list'>      409791
external_ids          facebook_id   <class 'NoneType'>  406517
                                    <class 'str'>         3274
                      imdb_id       <class 'NoneType'>   61376
                                    <class 'str'>       348415
                      instagram_id  <class 'NoneType'>  408572
                                    <class 'str'>         1219
                      twitter_id    <class 'NoneType'>  408145
                                    <class 'str'>         1646
genres                NotDict       <class 'list'>      409791
homepage              NotDict       <class 'NoneType'>  365709
                                    <class 'str'>        44082
id                    NotDict       <class 'int'>       409791
images                backdrops     <class 'list'>      409791
                      posters       <class 'list'>      409791
imdb_id               NotDict       <class 'NoneType'>   61376
                                    <class 'str'>       348415
...                                                        ...
overview              NotDict       <class 'str'>       409791
popularity            NotDict       <class 'float'>     409791
poster_path           NotDict       <class 'NoneType'>   84776
                                    <class 'str'>       325015
production_companies  NotDict       <class 'list'>      409791
production_countries  NotDict       <class 'list'>      409791
recommendations       page          <class 'int'>       409791
                      results       <class 'list'>      409791
                      total_pages   <class 'int'>       409791
                      total_results <class 'int'>       409791
release_date          NotDict       <class 'str'>       409791
release_dates         results       <class 'list'>      409791
revenue               NotDict       <class 'int'>       409791
reviews               page          <class 'int'>       409791
                      results       <class 'list'>      409791
                      total_pages   <class 'int'>       409791
                      total_results <class 'int'>       409791
runtime               NotDict       <class 'NoneType'>  120276
                                    <class 'int'>       289515
similar_movies        page          <class 'int'>       409791
                      results       <class 'list'>      409791
                      total_pages   <class 'int'>       409791
                      total_results <class 'int'>       409791
spoken_languages      NotDict       <class 'list'>      409791
status                NotDict       <class 'str'>       409791
tagline               NotDict       <class 'str'>       409791
title                 NotDict       <class 'str'>       409791
video                 NotDict       <class 'bool'>      409791
vote_average          NotDict       <class 'float'>     409791
vote_count            NotDict       <class 'int'>       409791

[63 rows x 1 columns]

It will be good to keep this document as an excel file (we will review it several times!)

In [0]:
!pip install openpyxl
import openpyxl

dfTypeCheck.to_excel('typesCheck.xlsx')

#And then we copy it to our Drive:
!cp typesCheck.xlsx /content/drive/"My Drive"

    100% |████████████████████████████████| 174kB 13.2MB/s 
  Running setup.py bdist_wheel for openpyxl ... - \ error
  Complete output from command /usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-zzxqb3v9/openpyxl/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-nm3cjpgo --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/openpyxl
  copying openpyxl/_constants.py -> build/lib/openpyxl
  copying openpyxl/__init__.py -> build/lib/openpyxl
  copying openpyxl/conftest.py -> build/lib/openpyxl
  creating build/lib/openpyxl/formula
  copying openpyxl/formula/translate.py -> build/lib/openpyxl/formula
  copying openpyxl/formula/__init__.py -> build/lib/openpyxl/formula
  copying openpyxl/formula/tokenizer.py -> build/lib/openpyxl/formula
  creating build/lib

##3.03 Importing json files to a pandas dataframe.

After analizing the list of fields in **`dfTypeCheck`** and the documentation of [The Movie Database API](https://developers.themoviedb.org/3/movies/get-movie-details), we select the fields we will import. Getting the whole information of all fields could take most of the available memory. So this is an aspect to take into account. However, we will take more fields than we will finally use, as this will allow us to enhance the model we are going to develop in the future.

Thus we create a generator function to import the selected fields. Using a generator function proves to be the most efficient way with regards to memory. While importing the files, we convert the first level of dictionaries into columns of a dataframe, take the fields we find interesting and drop the others. We exclude "adult films" as this is out of our scope. We collect all movies information in **`dfMovies`** pandas dataframe.

In [0]:
def moviesIterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:
                    data = ujson.load(o)
                
                if intI % 10000 == 0:
                    #Just to control how it goes:
                    print(intI)
                    
                result = {"titles": data["alternative_titles"]["titles"], 
                       "budget": data["budget"], "credits_cast": data["credits"]["cast"], "credits_crew": data["credits"]["crew"],
                       "genres": data["genres"], "movie_id": data["id"], "imdb_id": data["imdb_id"],
                       "keywords": data["keywords"]["keywords"], "original_language": data["original_language"],
                       "original_title": data["original_title"], "overview": data["overview"],
                       "popularity": data["popularity"], 
                       "production_companies": data["production_companies"], "production_countries": data["production_countries"],
                       "release_date": data["release_date"], "release_dates": data["release_dates"]["results"],
                       "revenue": data["revenue"], 
                       "reviews_page": data["reviews"]["page"],
                       "reviews_results": data["reviews"]["results"],
                       "reviews_total_pages": data["reviews"]["total_pages"],
                       "reviews_total_results": data["reviews"]["total_results"],
                       "runtime": data["runtime"],
                       "spoken_languages": data["spoken_languages"], "status": data["status"],
                       "tagline": data["tagline"], "title": data["title"], 
                       "vote_average": data["vote_average"], "vote_count": data["vote_count"]}
                    
                #We check in dfTypeCheck that "belongs_to_collection" is sometimes null. This is why we need to deal with it separately:
                if data["belongs_to_collection"] is None:
                    result["belongs_to_collection_id"] = None
                    result["belongs_to_collection_name"] = None
                else:
                    result["belongs_to_collection_id"] = data["belongs_to_collection"]["id"]
                    result["belongs_to_collection_name"] = data["belongs_to_collection"]["name"]
                    
                #We tale only non adult films:
                if data["adult"] == False:
                    yield result

In [0]:
path = "./themoviedb_data/movies"
dfMovies = pd.DataFrame()
tic=timeit.default_timer()
dfMovies = pd.DataFrame(moviesIterator(path))
toc=timeit.default_timer()
print((toc - tic)/60, ' min') #elapsed time in minutes

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


323.37504019900007

In [0]:
dfMovies.shape

(393239, 30)

In [0]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'credits_cast', 'credits_crew', 'genres', 'imdb_id', 'keywords',
       'movie_id', 'original_language', 'original_title', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'release_dates', 'revenue', 'reviews_page',
       'reviews_results', 'reviews_total_pages', 'reviews_total_results',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count'],
      dtype='object')

Considering the way we got the json files, we can be sure that **`movie_id`** will be the primary key for **`dfMovies`**. But it is good to make sure about it!

In [0]:
dfMovies['movie_id'].dtypes

In [0]:
dfMovies['movie_id'].isnull().sum()

In [0]:
dfMovies['movie_id'].duplicated().sum()

Good: **`movie_id`** is numeric with no duplicates or nulls.

##3.04 Tidying dfMovies dataframe.

Some of the columns in dfMovies have their elements made by lists of dictionaries. That is the case of credits_cast:

In [0]:
np.sort(dfMovies['credits_cast'].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 120, 121, 122, 123, 124, 125, 127, 128, 130, 131, 133,
       134, 135, 136, 137, 139, 140, 142, 143, 145, 146, 151, 152, 153,
       154, 158, 159, 164, 165, 168, 170, 171, 172, 183, 185, 194, 208,
       213, 224, 268, 313])

In [0]:
#One example:
type(dfMovies[dfMovies.index ==76341]['credits_cast'].iloc[0])

list

In [0]:
dfMovies[dfMovies.index ==76341]['credits_cast'].map(len)

76341    2
Name: credits_cast, dtype: int64

In [0]:
dfMovies[dfMovies.index ==76341]['credits_cast'].iloc[0]

[{'cast_id': 0,
  'character': 'Herself',
  'credit_id': '543aec2dc3a368198c0038d0',
  'gender': 1,
  'id': 1459,
  'name': 'Tina Turner',
  'order': 1,
  'profile_path': '/yhA10YPoQwGWtvd4TUm4X6tkv1x.jpg'},
 {'cast_id': 1,
  'character': 'Musician (Guitars, Vocals)',
  'credit_id': '543aec810e0a26499e003a43',
  'gender': 0,
  'id': 580258,
  'name': 'John Miles',
  'order': 2,
  'profile_path': None}]

As we see in [The Movie Database API](https://developers.themoviedb.org/3/movies/get-movie-credits) documentation, these keys are the same in all credits casts:

In [0]:
dfMovies[dfMovies.index ==76341]['credits_cast'].iloc[0][0].keys()

dict_keys(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'profile_path'])

In [0]:
dfMovies[dfMovies.index ==76341]['credits_cast'].iloc[0][0]

{'cast_id': 0,
 'character': 'Herself',
 'credit_id': '543aec2dc3a368198c0038d0',
 'gender': 1,
 'id': 1459,
 'name': 'Tina Turner',
 'order': 1,
 'profile_path': '/yhA10YPoQwGWtvd4TUm4X6tkv1x.jpg'}

So every element in credits_cast is a list with different lengths, where every element is a dictionary, all with the same keys. We would need to transform this structure in a way more easy to deal with. We could do it with a function as the following:

In [0]:
#Firstly we need movies ids to be the index of dfMovies.
dfMovies.index = dfMovies['movie_id']

In [0]:
#And so we could define this function to deal with credits_cast:
def cast_iterator():
    for id in dfMovies.index:
        for dict in dfMovies[dfMovies.index == id]['credits_cast'].iloc[0]:
            output = {'movie_id': id, 'cast_id': dict['cast_id'], 'character': dict['character'], 'gender': dict['gender'],
                     'id': dict['id'], 'name': dict['name'], 'order': dict['order']}
            yield output
#And it works properly if dfMovies[dfMovies.index == id]['credits_cast'].iloc[0] is an empty list!!

This function would create a new dataframe storing the information of credits_cast, where **`movie_id`** and **`id`** (the cast primary key) are expected to be primary keys. After creating this new dataframe, we could drop column **`credits_cast`** from **`dfMovies`**. 

But it would be interesting to make this fuction more flexible to deal with all columns in **`dfMovies`** that has similar structure. This new function should be able to manage the situation where the number and name of keys is different in every column with this kind of structure. That is **`columnIteratorMovies`**:

In [0]:
#This is the function we need, which is able to handle a variable number of keys!!
def columnIteratorMovies(columnName, *args):
    for id in dfMovies.index:
        for dict in dfMovies[dfMovies.index == id][columnName].iloc[0]:
            outputDict = {'movie_id': id}
            for key in args:
                outputDict[key] = dict.get(key,None)
            yield outputDict       

And we could automate as well the process of droping the columns in **`dfMovies`** once we have processed them and even perform a "sanity check" just to be sure the number of rows in the new dataframes are the expected ones. We will do all this with **`columnProcessesMovies`** function.

In [0]:
def columnProcessesMovies (columnName, *args):
    #I need to declare dfMovies as global as I will change it inside this function.
    global dfMovies
    
    dfOutput = pd.DataFrame()
    dfOutput = pd.DataFrame(columnIteratorMovies(columnName, *args))
    
    #Sanity check:
    dfCheck = pd.DataFrame(dfMovies[columnName].map(len).value_counts())
    dfCheck['calculation']=dfCheck[columnName]*dfCheck.index
    
    if dfOutput.shape[0] == dfCheck['calculation'].sum():
        dfMovies = dfMovies.drop(columnName, axis=1)
        return dfOutput
    else:
        return "Error in process"

Now we need to analyze which keys we would like to get from every column with this structure:

###CREDITS_CAST

In [0]:
columnName = 'credits_cast'

In [0]:
np.sort(dfMovies[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 120, 121, 122, 123, 124, 125, 127, 128, 130, 131, 133,
       134, 135, 136, 137, 139, 140, 142, 143, 145, 146, 151, 152, 153,
       154, 158, 159, 164, 165, 168, 170, 171, 172, 183, 185, 194, 208,
       213, 224, 268, 313])

In [0]:
#Pay attention to the number of 0's!
dfMovies[columnName].map(len).value_counts()

0      124820
1       27846
4       25515
5       24594
6       21337
3       19676
2       18525
7       17688
8       16379
9       13341
10      12699
11       9759
12       8586
15       8218
13       6861
14       6048
16       4352
17       3375
18       2834
19       2394
20       2161
21       1818
22       1481
23       1338
24       1106
25        952
26        878
27        758
28        724
29        574
        ...  
171         2
136         2
122         2
153         1
158         1
268         1
159         1
172         1
165         1
168         1
170         1
185         1
194         1
154         1
116         1
152         1
151         1
145         1
142         1
140         1
139         1
134         1
133         1
130         1
127         1
124         1
121         1
313         1
213         1
224         1
Name: credits_cast, Length: 160, dtype: int64

In [0]:
#Having a look to one example:
dfMovies[dfMovies["movie_id"]==76341][columnName].iloc[0]

[{'cast_id': 2,
  'character': 'Max Rockatansky',
  'credit_id': '52fe4933c3a368484e11f773',
  'gender': 2,
  'id': 2524,
  'name': 'Tom Hardy',
  'order': 0,
  'profile_path': '/4CR1D9VLWZcmGgh4b6kKuY2NOel.jpg'},
 {'cast_id': 35,
  'character': 'Imperator Furiosa',
  'credit_id': '560e8a2bc3a368681f00b80c',
  'gender': 1,
  'id': 6885,
  'name': 'Charlize Theron',
  'order': 1,
  'profile_path': '/aHOB9UMgSI7MlXF3GMZaVQXqfQg.jpg'},
 {'cast_id': 4,
  'character': 'Nux',
  'credit_id': '52fe4933c3a368484e11f77b',
  'gender': 2,
  'id': 3292,
  'name': 'Nicholas Hoult',
  'order': 2,
  'profile_path': '/27NbnWKzco1CCDHtR2AVVab8Dli.jpg'},
 {'cast_id': 21,
  'character': 'Immortan Joe',
  'credit_id': '55549e64c3a3682086002b85',
  'gender': 2,
  'id': 26060,
  'name': 'Hugh Keays-Byrne',
  'order': 3,
  'profile_path': '/lhYUcPimMMy7Ltp58NkB3RuJzSv.jpg'},
 {'cast_id': 22,
  'character': 'Slit',
  'credit_id': '55549e84c3a36820800028e6',
  'gender': 2,
  'id': 1056053,
  'name': 'Josh Helma

In [0]:
dfMovies[dfMovies["movie_id"]==76341][columnName].iloc[0][0]

{'cast_id': 2,
 'character': 'Max Rockatansky',
 'credit_id': '52fe4933c3a368484e11f773',
 'gender': 2,
 'id': 2524,
 'name': 'Tom Hardy',
 'order': 0,
 'profile_path': '/4CR1D9VLWZcmGgh4b6kKuY2NOel.jpg'}

In [0]:
dfMovies[dfMovies["movie_id"]==76341][columnName].iloc[0][0].keys()

dict_keys(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'profile_path'])

In [0]:
tic=timeit.default_timer()
dfMoviesCast = columnProcessesMovies (columnName, 'cast_id', 'character', 'gender', 'id', 'name', 'order')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

363.9738898020005

In [0]:
dfMoviesCast.shape

(2278079, 7)

In [0]:
dfMoviesCast.tail()

,cast_id,character,gender,id,movie_id,name,order
2278074,48,RAF Officer,0,62987,10079,Robert Sidaway,19
2278075,1,Himself,2,39657,32986,Stephen Hawking,0
2278076,3,Himself,2,241,32986,Arthur C. Clarke,2
2278077,8,Presenter,0,109886,32986,Magnus Magnusson,3
2278078,9,Himself,2,10293,32986,Carl Sagan,4


In [0]:
#Looking at our example:
dfMoviesCast[dfMoviesCast["movie_id"]==76341]

,cast_id,character,gender,id,movie_id,name,order
1756376,2,Max Rockatansky,2,2524,76341,Tom Hardy,0
1756377,35,Imperator Furiosa,1,6885,76341,Charlize Theron,1
1756378,4,Nux,2,3292,76341,Nicholas Hoult,2
1756379,21,Immortan Joe,2,26060,76341,Hugh Keays-Byrne,3
1756380,22,Slit,2,1056053,76341,Josh Helman,4
1756381,14,Rictus Erectus,2,24898,76341,Nathan Jones,5
1756382,8,Toast the Knowing,1,37153,76341,Zoë Kravitz,6
1756383,12,The Splendid Angharad,1,236048,76341,Rosie Huntington-Whiteley,7
1756384,13,Capable,1,98522,76341,Riley Keough,8
1756385,10,The Dag,1,1036288,76341,Abbey Lee,9


In [0]:
#Looking at dfMovies columns:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'credits_crew', 'genres', 'imdb_id', 'keywords', 'movie_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'release_dates', 'revenue', 'reviews_page', 'reviews_results',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count'],
      dtype='object')

In [0]:
dfMovies.shape

(393239, 29)

### CREDITS_CREW

In [0]:
columnName = 'credits_crew'

In [0]:
np.sort(dfMovies[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 173, 174, 175, 177, 178, 179, 180, 181, 183, 18

In [0]:
#Pay attention to the number of 0's!
dfMovies[columnName].map(len).value_counts()

1      106010
0       81188
2       56282
3       33892
4       22010
5       15923
6       12662
7       10474
8        8699
9        7161
10       6746
11       5133
12       4085
13       3284
14       2663
15       2151
16       1752
17       1379
18       1196
19        999
20        810
21        735
22        583
23        535
24        510
25        439
26        395
28        332
27        330
29        256
        ...  
305         1
324         1
323         1
365         1
321         1
167         1
166         1
421         1
416         1
417         1
300         1
183         1
209         1
298         1
208         1
318         1
203         1
457         1
286         1
201         1
198         1
197         1
340         1
291         1
143         1
334         1
295         1
188         1
187         1
171         1
Name: credits_crew, Length: 263, dtype: int64

In [0]:
#Looking at one example:
dfMovies[dfMovies["movie_id"]==76341][columnName].iloc[0]

[{'credit_id': '53d451210e0a262841002dd1',
  'department': 'Writing',
  'gender': 2,
  'id': 1027146,
  'job': 'Writer',
  'name': 'Nick Lathouris',
  'profile_path': '/tlkHM8TUcKZFcsS7rGuShkCHeU.jpg'},
 {'credit_id': '52fe4933c3a368484e11f76f',
  'department': 'Directing',
  'gender': 2,
  'id': 20629,
  'job': 'Director',
  'name': 'George Miller',
  'profile_path': '/4FyQESSzuu1VkPup9CmMjdnUyhv.jpg'},
 {'credit_id': '52fe4933c3a368484e11f78d',
  'department': 'Production',
  'gender': 2,
  'id': 11651,
  'job': 'Producer',
  'name': 'Doug Mitchell',
  'profile_path': '/oWMx5Mz4xjADeupXHgNBT1TZwut.jpg'},
 {'credit_id': '53d451680e0a26283b002dbb',
  'department': 'Writing',
  'gender': 2,
  'id': 20629,
  'job': 'Writer',
  'name': 'George Miller',
  'profile_path': '/4FyQESSzuu1VkPup9CmMjdnUyhv.jpg'},
 {'credit_id': '552c5f3b92514102250002cb',
  'department': 'Visual Effects',
  'gender': 2,
  'id': 1451274,
  'job': 'Visual Effects',
  'name': 'Pablo Calvillo',
  'profile_path': '/h

In [0]:
dfMovies[dfMovies["movie_id"]==76341][columnName].iloc[0][0]

{'credit_id': '53d451210e0a262841002dd1',
 'department': 'Writing',
 'gender': 2,
 'id': 1027146,
 'job': 'Writer',
 'name': 'Nick Lathouris',
 'profile_path': '/tlkHM8TUcKZFcsS7rGuShkCHeU.jpg'}

In [0]:
dfMovies[dfMovies["movie_id"] ==76341][columnName].iloc[0][0].keys()

dict_keys(['credit_id', 'department', 'gender', 'id', 'job', 'name', 'profile_path'])

In [0]:
tic=timeit.default_timer()
dfMoviesCrew = columnProcessesMovies (columnName, 'department', 'gender', 'id', 'job', 'name')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

352.9851637080028

In [0]:
dfMoviesCrew.shape

(1548760, 6)

In [0]:
dfMoviesCrew.tail()

,department,gender,id,job,movie_id,name
1548755,Production,2,91499,Producer,32986,Kevin Dowling
1548756,Production,0,109883,Executive Producer,32986,Robert Southgate
1548757,Editing,0,109884,Editor,32986,Christopher Jeans
1548758,Directing,0,1067801,Director,521201,Ali Cotterill
1548759,Production,0,2031751,Producer,521201,Christa Orth


In [0]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'genres', 'imdb_id', 'keywords', 'movie_id', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'release_dates', 'revenue',
       'reviews_page', 'reviews_results', 'reviews_total_pages',
       'reviews_total_results', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'titles', 'vote_average', 'vote_count'],
      dtype='object')

In [0]:
dfMovies.shape

(393239, 28)

###GENRES

In [0]:
columnName = 'genres'

In [0]:
np.sort(dfMovies[columnName].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 13])

In [0]:
#Pay attention to the number of 0's!
dfMovies[columnName].map(len).value_counts()

1     147592
0     138441
2      68282
3      29517
4       7451
5       1573
6        330
7         47
9          2
8          2
13         1
11         1
Name: genres, dtype: int64

In [0]:
#Looking at one example:
dfMovies[dfMovies["movie_id"]==76341][columnName].iloc[0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 53, 'name': 'Thriller'}]

In [0]:
dfMovies[dfMovies["movie_id"]==76341][columnName].iloc[0][0]

{'id': 28, 'name': 'Action'}

In [0]:
dfMovies[dfMovies["movie_id"]==76341][columnName].iloc[0][0].keys()

dict_keys(['id', 'name'])

In [0]:
tic=timeit.default_timer()
dfMoviesGenres = columnProcessesMovies (columnName, 'id', 'name')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

345.3144972789996

In [0]:
dfMoviesGenres.shape

(412743, 3)

In [0]:
dfMoviesGenres.tail()

,id,movie_id,name
412738,27,533049,Horror
412739,9648,533049,Mystery
412740,18,10079,Drama
412741,99,32986,Documentary
412742,99,521201,Documentary


In [0]:
dfMovies.shape

(393239, 27)

In [0]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'imdb_id', 'keywords', 'movie_id', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'release_dates', 'revenue',
       'reviews_page', 'reviews_results', 'reviews_total_pages',
       'reviews_total_results', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'titles', 'vote_average', 'vote_count'],
      dtype='object')

###KEYWORDS

In [0]:
columnName = 'keywords'

In [0]:
np.sort(dfMovies[columnName].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 54, 60,
       63, 80])

In [0]:
#Pay attention to the number of 0's!
dfMovies[columnName].map(len).value_counts()

0     262347
1      47294
2      27199
3      18507
4      12028
5       7836
6       4761
7       3161
8       2185
9       1584
10      1282
11      1035
12       846
13       653
14       507
15       406
16       332
17       254
18       226
20       190
19       183
21       106
22        70
23        57
24        39
25        24
27        23
26        20
29        13
28        11
31         9
37         5
30         5
32         5
33         4
38         4
41         3
35         3
36         3
43         2
42         2
39         2
40         2
34         2
47         1
60         1
54         1
48         1
63         1
46         1
45         1
44         1
80         1
Name: keywords, dtype: int64

In [0]:
#Looking at one example:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0]

[{'id': 2964, 'name': 'future'},
 {'id': 3713, 'name': 'chase'},
 {'id': 4458, 'name': 'post-apocalyptic'},
 {'id': 4565, 'name': 'dystopia'},
 {'id': 5657, 'name': 'australia'},
 {'id': 10084, 'name': 'rescue'},
 {'id': 10349, 'name': 'survival'},
 {'id': 10562, 'name': 'on the run'},
 {'id': 155499, 'name': 'convoy'},
 {'id': 190954, 'name': 'peak oil'},
 {'id': 212516, 'name': 'dark future'}]

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0]

{'id': 2964, 'name': 'future'}

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0].keys()

dict_keys(['id', 'name'])

In [0]:
tic=timeit.default_timer()
dfMoviesKeywords = columnProcessesMovies (columnName, 'id', 'name')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

346.86361952100197

In [0]:
dfMoviesKeywords.shape

(414659, 3)

In [0]:
dfMoviesKeywords.tail()

,id,movie_id,name
414654,9649,32986,god
414655,9739,32986,extraterrestrial
414656,9882,32986,space
414657,156810,32986,science
414658,161288,32986,existence


In [0]:
dfMovies.shape

(393239, 26)

In [0]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'imdb_id', 'movie_id', 'original_language', 'original_title',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'release_dates', 'revenue',
       'reviews_page', 'reviews_results', 'reviews_total_pages',
       'reviews_total_results', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'titles', 'vote_average', 'vote_count'],
      dtype='object')

In [0]:
#How frequent are the values?
dfMoviesKeywords['name'].value_counts()

woman director               13360
short                         8254
murder                        3199
independent film              2933
silent film                   2652
musical                       2500
based on novel or book        2373
sport                         2329
biography                     1952
sex                           1589
music                         1577
revenge                       1479
philippines                   1432
pinoy                         1420
experimental                  1409
love                          1392
stand-up comedy               1243
concert                       1193
world war ii                  1192
martial arts                  1156
live concert                  1137
violence                      1119
family                        1016
lgbt                          1012
parent child relationship      996
suspense                       970
christmas                      915
police                         903
anime               

###PRODUCTION_COMPANIES

In [0]:
columnName = 'production_companies'

In [0]:
np.sort(dfMovies[columnName].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 25, 26])

In [0]:
#Pay attention to the number of 0's!
dfMovies[columnName].map(len).value_counts()

0     234413
1     111180
2      28234
3      11687
4       3834
5       1739
6        887
7        487
8        267
9        196
10        86
11        59
12        50
13        26
14        21
16        21
15        12
17        10
20         7
19         6
22         5
18         4
21         3
26         3
25         2
Name: production_companies, dtype: int64

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0]

[{'id': 79,
  'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
  'name': 'Village Roadshow Pictures',
  'origin_country': 'US'},
 {'id': 2537,
  'logo_path': None,
  'name': 'Kennedy Miller Productions',
  'origin_country': 'AU'},
 {'id': 174,
  'logo_path': '/ky0xOc5OrhzkZ1N6KyUxacfQsCk.png',
  'name': 'Warner Bros. Pictures',
  'origin_country': 'US'}]

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0]

{'id': 79,
 'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
 'name': 'Village Roadshow Pictures',
 'origin_country': 'US'}

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0].keys()

dict_keys(['id', 'logo_path', 'name', 'origin_country'])

In [0]:
tic=timeit.default_timer()
dfMoviesProductionComp = columnProcessesMovies (columnName, 'id', 'name', 'origin_country')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

340.8884366470011

In [0]:
dfMoviesProductionComp.shape

(243425, 4)

In [0]:
dfMoviesProductionComp.tail()

,id,movie_id,name,origin_country
243420,856,336728,Wild Bunch,FR
243421,103439,518485,Studio Batu,
243422,3324,201670,British Broadcasting Corporation,GB
243423,5857,10079,Kingsway Films,
243424,5238,32986,ITV Central,GB


In [0]:
dfMovies.shape

(393239, 25)

In [0]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'imdb_id', 'movie_id', 'original_language', 'original_title',
       'overview', 'popularity', 'production_countries', 'release_date',
       'release_dates', 'revenue', 'reviews_page', 'reviews_results',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count'],
      dtype='object')

###PRODUCTION_COUNTRIES

In [0]:
columnName = 'production_countries'

In [0]:
np.sort(dfMovies[columnName].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 20,
       25, 26])

In [0]:
#Pay attention to the number of 0's!
dfMovies[columnName].map(len).value_counts()

1     199184
0     172114
2      16736
3       3688
4       1008
5        340
6         95
7         32
8         15
9          7
11         6
10         4
12         3
25         2
14         1
15         1
16         1
20         1
26         1
Name: production_countries, dtype: int64

In [0]:
#Looking at one example:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0]

[{'iso_3166_1': 'AU', 'name': 'Australia'},
 {'iso_3166_1': 'US', 'name': 'United States of America'}]

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0]

{'iso_3166_1': 'AU', 'name': 'Australia'}

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0].keys()

dict_keys(['iso_3166_1', 'name'])

In [0]:
tic=timeit.default_timer()
dfMoviesProductionCountries = columnProcessesMovies (columnName, 'iso_3166_1', 'name')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

348.04032126200036

In [0]:
dfMoviesProductionCountries.shape

(250712, 3)

In [0]:
dfMoviesProductionCountries.tail()

,iso_3166_1,movie_id,name
250707,GB,201670,United Kingdom
250708,FR,64427,France
250709,GB,10079,United Kingdom
250710,GB,32986,United Kingdom
250711,US,521201,United States of America


In [0]:
dfMovies.shape

(393239, 24)

In [0]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'imdb_id', 'movie_id', 'original_language', 'original_title',
       'overview', 'popularity', 'release_date', 'release_dates', 'revenue',
       'reviews_page', 'reviews_results', 'reviews_total_pages',
       'reviews_total_results', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'titles', 'vote_average', 'vote_count'],
      dtype='object')

###REVIEWS_RESULTS

In [0]:
columnName = 'reviews_results'

In [0]:
np.sort(dfMovies[columnName].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10])

In [0]:
#Pay attention to the number of 0's!
dfMovies[columnName].map(len).value_counts()

0     387174
1       5159
2        619
3        166
4         64
6         19
5         19
7         10
8          5
10         4
Name: reviews_results, dtype: int64

In [0]:
#Looking at one example:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0]

[{'author': 'Phileas Fogg',
  'content': "Fabulous action movie. Lots of interesting characters. They don't make many movies like this. The whole movie from start to finish was entertaining I'm looking forward to seeing it again. I definitely recommend seeing it.",
  'id': '55660928c3a3687ad7001db1',
  'url': 'https://www.themoviedb.org/review/55660928c3a3687ad7001db1'},
 {'author': 'Andres Gomez',
  'content': 'Good action movie with a decent script for the genre. The photography is really good too but, in the end, it is quite repeating itself from beginning to end and the stormy OST is exhausting.',
  'id': '55732a53925141456e000639',
  'url': 'https://www.themoviedb.org/review/55732a53925141456e000639'},
 {'author': 'extoix',
  'content': 'Awesome movie!  WITNESS ME will stick with me forever!',
  'id': '55edd26792514106d600e380',
  'url': 'https://www.themoviedb.org/review/55edd26792514106d600e380'},
 {'author': 'Cineport',
  'content': 'Fantastic action that makes up for some plot

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0]

{'author': 'Phileas Fogg',
 'content': "Fabulous action movie. Lots of interesting characters. They don't make many movies like this. The whole movie from start to finish was entertaining I'm looking forward to seeing it again. I definitely recommend seeing it.",
 'id': '55660928c3a3687ad7001db1',
 'url': 'https://www.themoviedb.org/review/55660928c3a3687ad7001db1'}

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0].keys()

dict_keys(['author', 'content', 'id', 'url'])

In [0]:
tic=timeit.default_timer()
dfMoviesReviewsResults = columnProcessesMovies (columnName, 'author', 'content', 'id')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

377.1782996969996

In [0]:
dfMoviesReviewsResults.shape

(7510, 4)

In [0]:
dfMoviesReviewsResults.tail()

,author,content,id,movie_id
7505,tweissel,Watching Frank Ferrante become Groucho Marx wa...,513771c2760ee33a32152b85,174011
7506,Rocketeer Raccoon,I first learned about Hyperspace a.k.a. Gremlo...,5944601c9251413fdd002cd0,159411
7507,Andres Gomez,Unsuccessful children movie trying to follow t...,50c3e6cf760ee33d5a000a64,37686
7508,Leilah Maneveld,Firstly this movie is amazing!!!!!!!!!!!!! I ...,5730605bc3a36858790001a0,350555
7509,John Chard,A Gift of Cochise.\r\n\r\nHondo and the Apache...,589a6020c3a3683aa3000501,150486


In [0]:
dfMovies.shape

(393239, 21)

In [0]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'original_language', 'original_title', 'overview', 'popularity',
       'release_date', 'release_dates', 'revenue', 'reviews_page',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count'],
      dtype='object')

###SPOKEN_LANGUAGES

In [0]:
columnName = 'spoken_languages'

In [0]:
np.sort(dfMovies[columnName].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 19, 25])

In [0]:
#Pay attention to the number of 0's!
dfMovies[columnName].map(len).value_counts()

1     205480
0     165405
2      16763
3       3920
4       1112
5        366
6        116
7         37
8         18
9         12
10         2
11         2
12         2
19         1
13         1
14         1
25         1
Name: spoken_languages, dtype: int64

In [0]:
#Looking at one exmple:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0]

[{'iso_639_1': 'en', 'name': 'English'}]

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0]

{'iso_639_1': 'en', 'name': 'English'}

In [0]:
dfMovies[dfMovies['movie_id']==76341][columnName].iloc[0][0].keys()

dict_keys(['iso_639_1', 'name'])

In [0]:
tic=timeit.default_timer()
dfMoviesSpokenLang = columnProcessesMovies (columnName, 'iso_639_1', 'name')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

380.22373138200055

In [0]:
dfMoviesSpokenLang.shape

(258388, 3)

In [0]:
dfMoviesSpokenLang.tail()

,iso_639_1,movie_id,name
258383,fr,10970,Français
258384,xx,193828,No Language
258385,el,57578,ελληνικά
258386,en,57578,English
258387,ko,350487,한국어/조선말


In [0]:
dfMovies.shape

(393239, 20)

In [0]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'original_language', 'original_title', 'overview', 'popularity',
       'release_date', 'release_dates', 'revenue', 'reviews_page',
       'reviews_total_pages', 'reviews_total_results', 'runtime', 'status',
       'tagline', 'title', 'titles', 'vote_average', 'vote_count'],
      dtype='object')

###RELEASE_DATES

In [0]:
np.sort(dfMovies['release_dates'].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 75, 77, 78, 80, 81, 84, 85, 86])

In [0]:
#Pay attention to the number of 0's!
dfMovies['release_dates'].map(len).value_counts()

1     316873
0      31136
2      25920
3       7630
4       3609
5       2045
6       1295
7        839
8        674
9        475
10       352
11       283
12       238
13       180
14       153
15       121
16       102
17        93
18        77
19        61
21        54
24        54
20        47
23        46
22        39
41        38
27        31
29        31
40        31
28        30
       ...  
49        12
52        11
55         9
65         9
58         9
61         9
53         8
57         8
51         8
60         8
73         7
67         7
63         6
75         6
62         6
59         6
69         6
78         4
68         4
70         4
77         3
72         3
64         2
71         2
66         2
80         2
81         2
86         2
85         1
84         1
Name: release_dates, Length: 82, dtype: int64

In [0]:
#Looking at one example:
dfMovies[dfMovies['movie_id']==76341]['release_dates'].iloc[0]
#One of the values is a dictionary again!

[{'iso_3166_1': 'ES',
  'release_dates': [{'certification': '16',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-15T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'IN',
  'release_dates': [{'certification': 'A',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-15T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'ZA',
  'release_dates': [{'certification': '10-12PG V',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-15T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'MX',
  'release_dates': [{'certification': '',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-14T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'IL',
  'release_dates': [{'certification': '',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-14T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'BD',
  'release_dates': [{'certification': '',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-22T00:00:00.000Z',
    

In [0]:
dfMovies[dfMovies['movie_id']==76341]['release_dates'].iloc[0][0]

{'iso_3166_1': 'ES',
 'release_dates': [{'certification': '16',
   'iso_639_1': '',
   'note': '',
   'release_date': '2015-05-15T00:00:00.000Z',
   'type': 3}]}

In [0]:
dfMovies[dfMovies['movie_id']==76341]['release_dates'].iloc[0][0].keys()

dict_keys(['iso_3166_1', 'release_dates'])

In [0]:
#But in this case, one of the keys (note), is not present in all situations. We have an example here:
dfMovies[dfMovies['movie_id']==76341]['release_dates'].iloc[0][33]

#https://developers.themoviedb.org/3/movies/get-movie-release-dates
#Release dates support different types:
#1: Premiere
#2: Theatrical (limited)
#3: Theatrical
#4: Digital
#5: Physical
#6: TV

{'iso_3166_1': 'SE',
 'release_dates': [{'certification': '15',
   'iso_639_1': '',
   'note': '',
   'release_date': '2015-05-14T00:00:00.000Z',
   'type': 3},
  {'certification': '15',
   'iso_639_1': '',
   'release_date': '2015-10-05T00:00:00.000Z',
   'type': 5},
  {'certification': '15',
   'iso_639_1': '',
   'note': '4K Ultra HD release',
   'release_date': '2016-04-12T00:00:00.000Z',
   'type': 5}]}

In [0]:
#According to The Movie Database API documentation, these are the fields available:
dfMovies[dfMovies.index ==76341]['release_dates'].iloc[0][33]['release_dates'][0].keys()

dict_keys(['certification', 'iso_639_1', 'note', 'release_date', 'type'])

In [0]:
#But sometimes 'note' is not available:
dfMovies[dfMovies.index ==76341]['release_dates'].iloc[0][33]['release_dates'][1].keys()
#This won´t be a big problem, as we won´t use it.

dict_keys(['certification', 'iso_639_1', 'release_date', 'type'])

In [0]:
#How different is the release_date for this movie from the relese_dates in this column?
dfMovies[dfMovies.index ==76341]['release_date'].iloc[0]

'2015-05-13'

In [0]:
tic=timeit.default_timer()
dfMoviesReleaseDates1 = columnProcessesMovies ('release_dates', 'iso_3166_1', 'release_dates')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

357.956477577005

In [0]:
dfMoviesReleaseDates1.shape

(502009, 3)

In [0]:
dfMoviesReleaseDates1.tail()

,iso_3166_1,movie_id,release_dates
502004,US,57578,"[{'certification': 'R', 'iso_639_1': '', 'note..."
502005,US,113279,"[{'certification': '', 'iso_639_1': '', 'note'..."
502006,KR,350487,"[{'certification': '12', 'iso_639_1': '', 'not..."
502007,IN,261702,"[{'certification': 'U', 'iso_639_1': '', 'note..."
502008,GB,283685,"[{'certification': '', 'iso_639_1': '', 'note'..."


So we still need to create a new function to deal with **`release_dates`** column from **`release_dates`** column in dfMovie:

In [0]:
def columnIteratorReleaseDates(*args):
    for id in dfMoviesReleaseDates1.index:
        for dict in dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['release_dates'].iloc[0]:
            outputDict = {'movie_id': dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['movie_id'].iloc[0],
                         'iso_3166_1': dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['iso_3166_1'].iloc[0]}
            for key in args:
                outputDict[key] = dict.get(key,None)
            yield outputDict            

In [0]:
#The execution time is much longer in this case (about 30minutes!)
tic=timeit.default_timer()
dfMoviesReleaseDates = pd.DataFrame(columnIteratorReleaseDates('certification', 'iso_639_1', 'release_date', 'type'))
toc=timeit.default_timer()
toc - tic #elapsed time in seconds. 


1442.4042638819956

In [0]:
dfMoviesReleaseDates.shape

(517722, 6)

In [0]:
dfMoviesReleaseDates.tail()

,certification,iso_3166_1,iso_639_1,movie_id,release_date,type
517717,R,US,,57578,1978-05-12T00:00:00.000Z,3
517718,,US,,113279,1999-01-01T00:00:00.000Z,3
517719,12,KR,,350487,1979-08-11T00:00:00.000Z,3
517720,U,IN,,261702,1967-01-01T00:00:00.000Z,3
517721,,GB,,283685,1912-01-02T00:00:00.000Z,3


In [0]:
#Having a look to my example:
dfMoviesReleaseDates[(dfMoviesReleaseDates['movie_id']==76341) & (dfMoviesReleaseDates['iso_3166_1']=='SE')]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,certification,iso_3166_1,iso_639_1,movie_id,release_date,type
441234,15,SE,,76341,2015-05-14T00:00:00.000Z,3
441235,15,SE,,76341,2015-10-05T00:00:00.000Z,5
441236,15,SE,,76341,2016-04-12T00:00:00.000Z,5


In [0]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'original_language', 'original_title', 'overview', 'popularity',
       'release_date', 'revenue', 'reviews_page', 'reviews_total_pages',
       'reviews_total_results', 'runtime', 'status', 'tagline', 'title',
       'titles', 'vote_average', 'vote_count'],
      dtype='object')